In [1]:
# Library Import

import os
import sys
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\My_Design\\Data_Science\\Data Science\\Data Science Notes\\Machine Learning\\Projects\\End-to-End Projects\\End-to-End-Machine-Learning-Project-with-Deployment'

In [10]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/kunalkalkotwar21/End-to-End-Machine-Learning-Project-with-Deployment.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='kunalkalkotwar21'
os.environ['MLFLOW_TRACKING_PASSWORD']='e493d450b8e9059d8c01a75ebb5b404bb27528b1'

In [4]:
# Perparing the model evaluation entities [config_entity]

from dataclasses import dataclass
from pathlib import Path

# Below is the data class for Model Evaluation configuration
# This class is used to define the configuration for Model Evaluation in a machine learning pipeline.
@dataclass(frozen=True)
class ModelEvaluationConfig:
    """Model Evaluation Configuration"""
    # Define the attributes of the class
    root_dir: Path
    model_path: Path
    test_data_path: Path
    report_path: Path
    all_param: dict
    target_column: str
    mlflow_tracking_uri: str

In [5]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    """Configuration Manager"""
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        """Initialize Configuration Manager"""
        # Initialize the ConfigurationManager with file paths for configuration, parameters, and schema.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)



        # Create directories for artifacts and data ingestion
        create_directories([self.config.artifacts_root])  # Create the artifacts root directory

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """Get Model Evaluation Configuration"""
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN["name"]
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            test_data_path = config.test_data_path,
            report_path = config.report_path,
            all_param = params,
            target_column = schema,
            mlflow_tracking_uri = 'https://dagshub.com/kunalkalkotwar21/End-to-End-Machine-Learning-Project-with-Deployment.mlflow'
            )
        
        return model_evaluation_config

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from urllib.parse import urlparse
import mlflow
import joblib

In [8]:
class ModelEvaluation:
    """Model Evaluation"""
    def __init__(self, config: ModelEvaluationConfig):
        """Initialize Model Evaluation"""
        self.config = config

    def eval_report(self, actual, predict):
        """Evaluation Report"""
        r2 = r2_score(actual, predict)
        rmse = np.sqrt(mean_squared_error(actual, predict))
        mae = mean_absolute_error(actual, predict)

        return r2, rmse, mae
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        target_column = str(self.config.target_column)

        test_x = test_data.drop([target_column], axis=1)
        test_y = test_data[[target_column]]

        mlflow.set_registry_uri(self.config.mlflow_tracking_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            predicted_quantity = model.predict(test_x)
            (r2, rmse, mae) = self.eval_report(test_y, predicted_quantity)
            
            # Saving matrices as local
            scores = {
                "r2": r2,
                "rmse": rmse,
                "mae": mae
            }
            save_json(path = Path(self.config.report_path), data=scores)

            mlflow.log_params(self.config.all_param)
            
            # Logging metrics into mlflow
            mlflow.log_metrics({
                "r2": r2,
                "rmse": rmse,
                "mae": mae
            })

            # Model registry does not support local filesystem
            if tracking_uri_type_store != 'file':

                # Register the model in the MLflow Model Registry
                # There are other ways to use the Model Registry, which depends on the use case.
                # Please refer to the MLflow documentation for more details.
                # https://www.mlflow.org/docs/latest/model-registry.html#registering-a-model
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")        

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlflow()
    print("Model evaluation completed successfully.")
except Exception as e:
    raise e

[2025-05-10 00:31:32,843: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-10 00:31:32,847: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-10 00:31:32,855: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-10 00:31:32,859: INFO: common: created directory at: artifacts]
[2025-05-10 00:31:33,523: INFO: common: json file saved at: artifacts\model_evaluation\report.json]


2025/05/10 00:31:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2025/05/10 00:31:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 6
Created version '6' of model 'ElasticNetModel'.


🏃 View run shivering-skink-213 at: https://dagshub.com/kunalkalkotwar21/End-to-End-Machine-Learning-Project-with-Deployment.mlflow/#/experiments/0/runs/ceb2d25fa952447f9773b50717b5be49
🧪 View experiment at: https://dagshub.com/kunalkalkotwar21/End-to-End-Machine-Learning-Project-with-Deployment.mlflow/#/experiments/0
Model evaluation completed successfully.
